In [14]:
import polars as pl

# part1

In [205]:
reindeer_example = {
    "Comet": {
        "speed": 14,
        "max_travel_time": 10,
        "rest_time": 127
    },
    "Dancer": {
        "speed": 16,
        "max_travel_time": 11,
        "rest_time": 162
    }
}

In [206]:
print(reindeer_example["Comet"])

{'speed': 14, 'max_travel_time': 10, 'rest_time': 127}


In [207]:
schema = {
    "time": pl.Int64,
    "reindeer": str,
    "status": str,
    "time_travelled": pl.Int64,
    "rest_remaining": pl.Int64,
    "distance": pl.Int64,
}
time_range = pl.int_range(0, 100, eager=True).alias("time")
# print(time_range)

In [234]:
df = pl.DataFrame(schema=schema)

curr_status == "resting"
rest_remaining = 0
for i in range(1005):
    curr_time = i
    curr_reindeer = "Dancer" 
    reindeer_speed = reindeer_example[curr_reindeer]["speed"]
    max_travel_time = reindeer_example[curr_reindeer]["max_travel_time"]
    rest_time = reindeer_example[curr_reindeer]["rest_time"]

    filtered = df.filter(pl.col("reindeer").str.contains(curr_reindeer)).sort("time")
    time_so_far = filtered["time_travelled"].last() if filtered.height > 0 else 0
    dist_so_far = filtered["distance"].last() if filtered.height > 0 else 0
    rest_remaining = filtered["rest_remaining"].last() if filtered.height > 0 else 0
        
    if time_so_far == max_travel_time and curr_status == "flying":
        curr_status = "resting"
        rest_remaining = rest_time
    elif not rest_remaining > 0: 
        curr_status = "flying"
        curr_time_travelled = time_so_far + [1 if i > 0 else 0][0] 
        curr_dist = dist_so_far + [(1 * reindeer_speed) if i > 0 else 0][0]
    elif rest_remaining > 0:
        rest_remaining -=  1
    if rest_remaining == 0 and curr_status == "resting" and curr_time > 0:
        curr_time_travelled = 0
    curr_instance = pl.DataFrame({"time": curr_time, "reindeer": curr_reindeer, "status": curr_status, 
                                 "time_travelled": curr_time_travelled, "rest_remaining": rest_remaining, 
                                  "distance": curr_dist})
    df = pl.concat([df, curr_instance], how="vertical")

print(df)    

shape: (1_005, 6)
┌──────┬──────────┬─────────┬────────────────┬────────────────┬──────────┐
│ time ┆ reindeer ┆ status  ┆ time_travelled ┆ rest_remaining ┆ distance │
│ ---  ┆ ---      ┆ ---     ┆ ---            ┆ ---            ┆ ---      │
│ i64  ┆ str      ┆ str     ┆ i64            ┆ i64            ┆ i64      │
╞══════╪══════════╪═════════╪════════════════╪════════════════╪══════════╡
│ 0    ┆ Dancer   ┆ flying  ┆ 0              ┆ 0              ┆ 0        │
│ 1    ┆ Dancer   ┆ flying  ┆ 1              ┆ 0              ┆ 16       │
│ 2    ┆ Dancer   ┆ flying  ┆ 2              ┆ 0              ┆ 32       │
│ 3    ┆ Dancer   ┆ flying  ┆ 3              ┆ 0              ┆ 48       │
│ 4    ┆ Dancer   ┆ flying  ┆ 4              ┆ 0              ┆ 64       │
│ …    ┆ …        ┆ …       ┆ …              ┆ …              ┆ …        │
│ 1000 ┆ Dancer   ┆ resting ┆ 11             ┆ 44             ┆ 1056     │
│ 1001 ┆ Dancer   ┆ resting ┆ 11             ┆ 43             ┆ 1056     │
│ 1002 